# UNESCO Heritage Sites Risk Modeling - Data Exploration

## Notebook 01: Data Exploration and Summary Statistics

**Purpose**: Load and explore data from the PostGIS database, generate summary statistics, and create initial visualizations.

**Contents**:
1. Database connection and data loading
2. Heritage sites overview and statistics
3. Hazard data summary (OSM, climate, earthquakes, fires, floods)
4. Geographic coverage maps
5. Data quality assessment

In [ ]:
# Import required libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import text
import warnings

# Import project modules
import sys
sys.path.append('..')
from src.db.connection import get_session, engine
from config.settings import EUROPE_BBOX, EUROPE_ISO_CODES

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

## 1. Database Connection and Data Loading

In [ ]:
# Test database connection
session = get_session()

try:
    result = session.execute(text("SELECT PostGIS_Version();"))
    version = result.scalar()
    print(f"✓ Connected to PostgreSQL with PostGIS {version}")
finally:
    session.close()

In [ ]:
# Load heritage sites
query = """
SELECT 
    id, whc_id, name, country, category, date_inscribed,
    in_danger, area_hectares, elevation_m, coastal_distance_km,
    ST_X(geom) as longitude, ST_Y(geom) as latitude,
    ST_AsText(geom) as wkt_geom
FROM unesco_risk.heritage_sites
ORDER BY country, name;
"""

sites_df = pd.read_sql(query, engine)
print(f"Loaded {len(sites_df)} heritage sites")
sites_df.head()

In [ ]:
# Convert to GeoDataFrame for spatial operations
from shapely import wkt

sites_df['geometry'] = sites_df['wkt_geom'].apply(wkt.loads)
sites_gdf = gpd.GeoDataFrame(sites_df, geometry='geometry', crs='EPSG:4326')

print(f"GeoDataFrame created with {len(sites_gdf)} sites")
print(f"CRS: {sites_gdf.crs}")
print(f"\nBounding box: {sites_gdf.total_bounds}")

## 2. Heritage Sites Overview and Statistics

In [ ]:
# Basic statistics
print("=" * 60)
print("HERITAGE SITES SUMMARY")
print("=" * 60)
print(f"Total sites: {len(sites_df)}")
print(f"Countries: {sites_df['country'].nunique()}")
print(f"\nCategory distribution:")
print(sites_df['category'].value_counts())
print(f"\nIn danger: {sites_df['in_danger'].sum()} sites")
print(f"\nDate range: {sites_df['date_inscribed'].min()} - {sites_df['date_inscribed'].max()}")

In [ ]:
# Top 10 countries by site count
country_counts = sites_df['country'].value_counts().head(10)

plt.figure(figsize=(12, 6))
country_counts.plot(kind='bar', color='steelblue')
plt.title('Top 10 Countries by UNESCO Site Count', fontsize=14, fontweight='bold')
plt.xlabel('Country', fontsize=12)
plt.ylabel('Number of Sites', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Category distribution pie chart
category_counts = sites_df['category'].value_counts()

plt.figure(figsize=(8, 8))
plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%',
        startangle=90, colors=['#FF9999', '#66B2FF', '#99FF99'])
plt.title('UNESCO Sites by Category', fontsize=14, fontweight='bold')
plt.axis('equal')
plt.show()

In [ ]:
# Sites inscribed over time
year_counts = sites_df.groupby('date_inscribed').size()

plt.figure(figsize=(14, 6))
plt.plot(year_counts.index, year_counts.values, marker='o', linewidth=2, markersize=4)
plt.fill_between(year_counts.index, year_counts.values, alpha=0.3)
plt.title('UNESCO Sites Inscribed Over Time', fontsize=14, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Number of Sites', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Hazard Data Summary

In [ ]:
# Load data counts from all tables
table_counts = {}

tables = [
    'heritage_sites',
    'urban_features',
    'climate_events',
    'earthquake_events',
    'fire_events',
    'flood_zones',
    'risk_scores'
]

session = get_session()
try:
    for table in tables:
        result = session.execute(text(f"SELECT COUNT(*) FROM unesco_risk.{table};"))
        table_counts[table] = result.scalar()
finally:
    session.close()

print("=" * 60)
print("DATABASE TABLE COUNTS")
print("=" * 60)
for table, count in table_counts.items():
    print(f"{table:20s}: {count:,}")

In [ ]:
# Visualize table counts
plt.figure(figsize=(12, 6))
colors = plt.cm.Set3(range(len(table_counts)))
plt.bar(table_counts.keys(), table_counts.values(), color=colors)
plt.title('Database Table Record Counts', fontsize=14, fontweight='bold')
plt.xlabel('Table', fontsize=12)
plt.ylabel('Record Count', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yscale('log')  # Log scale for better visualization
plt.grid(True, axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Geographic Coverage Map

In [ ]:
# Simple scatter plot of sites
fig, ax = plt.subplots(figsize=(16, 10))

# Color by category
categories = sites_gdf['category'].unique()
colors = {'Cultural': 'red', 'Natural': 'green', 'Mixed': 'blue'}

for cat in categories:
    subset = sites_gdf[sites_gdf['category'] == cat]
    ax.scatter(subset['longitude'], subset['latitude'], 
               c=colors.get(cat, 'gray'), label=cat, alpha=0.6, s=30)

ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.set_title('UNESCO Heritage Sites Geographic Distribution', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Data Quality Assessment

In [ ]:
# Check for missing values
print("=" * 60)
print("DATA QUALITY - MISSING VALUES")
print("=" * 60)
missing = sites_df.isnull().sum()
missing_pct = (missing / len(sites_df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})

print(missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False))

In [ ]:
# Elevation statistics
if 'elevation_m' in sites_df.columns:
    print("\n" + "=" * 60)
    print("ELEVATION STATISTICS")
    print("=" * 60)
    print(sites_df['elevation_m'].describe())
    
    # Elevation histogram
    plt.figure(figsize=(12, 6))
    sites_df['elevation_m'].dropna().hist(bins=50, edgecolor='black')
    plt.xlabel('Elevation (meters)', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.title('Distribution of Site Elevations', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Area statistics
print("\n" + "=" * 60)
print("SITE AREA STATISTICS")
print("=" * 60)
print(sites_df['area_hectares'].describe())

# Top 10 largest sites
print("\nTop 10 Largest Sites:")
largest = sites_df.nlargest(10, 'area_hectares')[['name', 'country', 'area_hectares']]
print(largest.to_string(index=False))

## Summary

This notebook provided an overview of the UNESCO heritage sites data including:
- Database connection verification
- Site count and distribution statistics
- Geographic coverage visualization
- Data quality assessment

**Next Steps**: 
- Proceed to Notebook 02 for risk analysis
- Explore specific hazard data in more detail
- Investigate data quality issues if any were found